In [11]:
import numpy as np
import random
import math

# Definición de la función mapeo
def mapeo(x, y, nr, nc, cscala):
    r = max(0, min(nr - round(y / cscala), nr - 1))
    c = max(0, min(round(x / cscala), nc - 1))
    return r, c

# Función para cargar la matriz desde un archivo .npy
def cargar_matriz(file_path):
    return np.load(file_path)

# Función que implementa la búsqueda codiciosa
def busqueda_codiciosa(matriz, posicion_inicial, limite_diferencia=2.0):
    filas, columnas = matriz.shape
    x, y = posicion_inicial
    trayectoria = [(x, y)]  # Almacena la trayectoria del explorador
    altura_actual = matriz[x, y]
    print(f"Posición inicial: ({x}, {y}), Altura inicial: {altura_actual} metros")

    while True:
        # Obtener los vecinos válidos (8-vecinos)
        vecinos = [(x+dx, y+dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if (dx != 0 or dy != 0)]
        vecinos_validos = [(vx, vy) for vx, vy in vecinos if 0 <= vx < filas and 0 <= vy < columnas]

        # Seleccionar el vecino con menor altura cuya diferencia sea menor a 2 metros
        menor_vecino = None

        for vx, vy in vecinos_validos:
            altura_vecino = matriz[vx, vy]
            if altura_vecino < altura_actual and abs(altura_actual - altura_vecino) <= limite_diferencia:
                if menor_vecino is None or altura_vecino < matriz[menor_vecino[0], menor_vecino[1]]:
                    menor_vecino = (vx, vy)

        # Si no hay vecino válido, salir del bucle
        if menor_vecino is None:
            print("No se encontró un vecino con una altura menor y dentro del límite de diferencia.")
            break

        # Moverse al vecino con menor altura
        x, y = menor_vecino
        altura_actual = matriz[x, y]
        trayectoria.append((x, y))

        # Imprimir el movimiento
        print(f"Movimiento a: ({x}, {y}), Altura: {altura_actual} metros")

    return trayectoria, matriz[x, y]

# Función que implementa el recocido simulado
def recocido_simulado(matriz, posicion_inicial, limite_diferencia=2.0, temperatura_inicial=100, enfriamiento=0.99):
    filas, columnas = matriz.shape
    x, y = posicion_inicial
    trayectoria = [(x, y)]  # Almacena la trayectoria del explorador
    temperatura = temperatura_inicial
    altura_actual = matriz[x, y]
    print(f"Posición inicial: ({x}, {y}), Altura inicial: {altura_actual} metros")

    while temperatura > 1:
        # Obtener los vecinos válidos (8-vecinos)
        vecinos = [(x+dx, y+dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if (dx != 0 or dy != 0)]
        vecinos_validos = [(vx, vy) for vx, vy in vecinos if 0 <= vx < filas and 0 <= vy < columnas]

        # Seleccionar un vecino al azar
        vx, vy = random.choice(vecinos_validos)
        altura_vecino = matriz[vx, vy]

        # Si la nueva posición es mejor, moverse
        if altura_vecino < altura_actual and abs(altura_actual - altura_vecino) <= limite_diferencia:
            x, y = vx, vy
            altura_actual = matriz[x, y]
        else:
            # Aceptar un peor movimiento según la probabilidad del recocido simulado
            diferencia_altura = abs(altura_vecino - altura_actual)
            if diferencia_altura <= limite_diferencia and random.uniform(0, 1) < math.exp(-diferencia_altura / temperatura):
                x, y = vx, vy
                altura_actual = matriz[x, y]

        trayectoria.append((x, y))

        # Imprimir el movimiento
        print(f"Movimiento a: ({x}, {y}), Altura: {altura_actual} metros, Temperatura: {temperatura}")

        # Reducir la temperatura
        temperatura *= enfriamiento

    return trayectoria, matriz[x, y]

# Pruebas de los algoritmos en varias posiciones con el mapeo
def pruebas_algoritmos(file_path, cscala=10.045):
    # Cargar la matriz
    matriz = cargar_matriz(file_path)
    nr, nc = matriz.shape  # Número de filas y columnas en la matriz

    # Posiciones de prueba (en metros)
    posiciones_metros = [
        (3200, 5000),  # Posición cercana al centro
        (3700, 4800),  # Posición 1 cercana al fondo
        (6000, 6000),  # Posición 2 cercana al fondo
        (1000, 2000),  # Posición 1 lejana
        (3000, 9000),  # Posición 2 lejana
        (7000, 5000)   # Posición aleatoria
    ]

    for i, (pos_x, pos_y) in enumerate(posiciones_metros):
        # Usar la función mapeo para obtener los índices en la matriz
        posicion_inicial = mapeo(pos_x, pos_y, nr, nc, cscala)
        print(f"\n===== Prueba {i+1}: Posición Inicial (metros): ({pos_x}, {pos_y}) =====")

        print("\n===== Búsqueda Codiciosa =====")
        trayectoria_codiciosa, altura_final_codiciosa = busqueda_codiciosa(matriz, posicion_inicial)
        print(f"Búsqueda codiciosa finalizó en: {trayectoria_codiciosa[-1]} con una altura de: {altura_final_codiciosa} metros\n")

        print("===== Recocido Simulado =====")
        trayectoria_recocido, altura_final_recocido = recocido_simulado(matriz, posicion_inicial)
        print(f"Recocido simulado finalizó en: {trayectoria_recocido[-1]} con una altura de: {altura_final_recocido} metros")

# Llamar la función de prueba
file_path = '/content/crater_map.npy'  # Reemplaza con la ruta correcta
pruebas_algoritmos(file_path)



===== Prueba 1: Posición Inicial (metros): (3200, 5000) =====

===== Búsqueda Codiciosa =====
Posición inicial: (579, 319), Altura inicial: 19.622644042968968 metros
Movimiento a: (579, 318), Altura: 18.09316162109397 metros
Movimiento a: (579, 317), Altura: 16.88699951171897 metros
Movimiento a: (579, 316), Altura: 15.028312988281469 metros
Movimiento a: (580, 315), Altura: 13.053227539062718 metros
Movimiento a: (581, 314), Altura: 11.285727539062718 metros
Movimiento a: (582, 313), Altura: 9.371701660156468 metros
Movimiento a: (583, 312), Altura: 7.786967773437718 metros
Movimiento a: (584, 311), Altura: 6.731242675781468 metros
Movimiento a: (585, 310), Altura: 6.163105468750218 metros
Movimiento a: (584, 309), Altura: 5.372785644531469 metros
Movimiento a: (583, 308), Altura: 4.125615234375219 metros
Movimiento a: (584, 307), Altura: 3.3430371093752185 metros
Movimiento a: (585, 306), Altura: 3.1147460937502185 metros
Movimiento a: (585, 305), Altura: 2.757001953125218 metros
Mo